In [2]:
from QuakeAPI.DBQueries import *
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Lining Up Data 

We have two data sources: EMSC and USGS. Many earthquakes are reported by both places. We need a way to line up the quakes that are the same between the two sources. Unfortunately the time can be off by as much as several seconds, so they aren't exact matches. 


## First thought: K-neighbors

for sake of experimentation I'm going to pull out 2000 quakes from USGS and EMSC and then compare them. 

In [3]:
USGS_QUAKES = query_all('SELECT id, time, latitude, longitude, magnitude FROM USGS where magnitude >= 0;')
EMSC_QUAKES = query_all('SELECT id, time, latitude, longitude, magnitude FROM EMSC ORDER BY TIME DESC;')
len(USGS_QUAKES), len(EMSC_QUAKES)

(23460, 101999)

Data is in the structure:

'id': quake[0]

'place': quake[1]

'time': quake[2]

'lat': quake[3]

'lon': quake[4]

'mag': quake[5]

In [4]:
#there were NA values to get rid of 
df = pd.DataFrame(USGS_QUAKES)
df = df.dropna()
df = df.set_index(0)
edf = pd.DataFrame(EMSC_QUAKES).set_index(0)
edf.head()


,1,2,3,4
0,,,,
1,1586237049900,37.80,16.91,2.9
2,1586236321100,19.20,-155.48,2.2
3,1586236127700,33.51,-116.52,2.1
4,1586234922400,38.42,25.88,2.1
5,1586234581600,38.27,38.82,2.1


In [5]:
# instanciate the classifier
classifier = NearestNeighbors(n_neighbors=1)

In [6]:
# train the classifier 
classifier.fit(df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                 radius=1.0)

In [7]:
# run the classifier on all the data 
distance, index = classifier.kneighbors(edf)

In [8]:
#check the best match
distance.min()

6.348473394446884

In [ ]:
# well damn. There aren't any found. I should come back and try alternate algorithms
times = []
for i, dist in enumerate(distance):
    if dist < 20000:
        if np.absolute(USGS_QUAKES[index[i][0]][1]-EMSC_QUAKES[i][1]) < 1000:
            print(USGS_QUAKES[index[i][0]])
            print(EMSC_QUAKES[i])
            print()

In [10]:
# attempting to do some lining up by hand
df[df[4]>.4].sort_values(by=4).head(30)

,1,2,3,4
0,,,,
3323,1582881115290,38.837666,-122.804169,0.41
15680,1584854077700,33.495167,-116.788667,0.41
2823,1582996500100,33.596333,-116.799833,0.41
3627,1582829274500,35.857333,-117.675500,0.41
23875,1586415189480,33.496333,-116.506000,0.41
8031,1581863391890,44.531000,-111.102500,0.41
788,1583502664250,33.223667,-116.717167,0.41
1105,1583420748580,35.954333,-117.703833,0.41
5856,1582343147980,33.503000,-116.770167,0.41


In [11]:
EMSC_df = pd.DataFrame(EMSC_QUAKES)

In [12]:
EMSC_df.sort_values(by=4).head(30)

,0,1,2,3,4
946,947,1585911218000,45.61,15.21,0.4
883,884,1585928095000,45.75,14.05,0.4
839,840,1585940360000,46.09,13.68,0.4
433,434,1586076338500,45.87,15.98,0.5
231,232,1586142907400,45.90,15.98,0.5
780,781,1585966547000,45.95,14.10,0.6
836,837,1585941571000,45.70,14.42,0.6
1286,1287,1585805621100,45.89,15.96,0.6
59,60,1586213070700,47.53,9.27,0.7
599,600,1586019353900,46.64,9.60,0.7


## thought 2, use the history

well nothing so far has produced usable results. I think I'm going to try to line up quakes using the history funciton. 

In [13]:
import requests

In [14]:
requests.get('http://quake-ds-staging.herokuapp.com/history/USGS/0,0,3000').json()

KeyboardInterrupt: 

In [ ]:
requests.get('http://quake-ds-staging.herokuapp.com/history/EMSC/0,0,3000').json()

The first three quakes around the area are clearly the same quakes, so there are some in the database, the question is just finding them. 
I'm going to try to algorithmically go through all of the EMSC quakes and search for proximity of time. 

In [ ]:
for quake in EMSC_QUAKES:
    url = f'http://quake-ds-staging.herokuapp.com/history/USGS/{quake[3]},{quake[4]},500'
    responses = requests.get(url).json()['message']
    if len(responses) != 0:
        for response in responses:
            if np.absolute(response['time'] - quake[1]) < 2000:
                print('EMSC:', quake)
                print(response)

This proved to be wayyyyy to slow. probably because its calling the api 20000 times 

I'm going to go back and try alternate algorithms now.

In [15]:
# instanciate the classifier
classifier = NearestNeighbors(n_neighbors=1, algorithm='kd_tree')
# train the classifier 
classifier.fit(df)
# run the classifier on all the data 
distance, index = classifier.kneighbors(edf)
#check the best match
distance.min()

6.348473394446884

In [16]:
# instanciate the classifier
classifier = NearestNeighbors(n_neighbors=1, algorithm='auto')
# train the classifier 
classifier.fit(df)
# run the classifier on all the data 
distance, index = classifier.kneighbors(edf)
#check the best match
distance.min()

6.348473394446884

In [17]:
# instanciate the classifier
classifier = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
# train the classifier 
classifier.fit(df)
# run the classifier on all the data 
distance, index = classifier.kneighbors(edf)
#check the best match
distance.min()

6.348473394446884

In [18]:
# instanciate the classifier
classifier = NearestNeighbors(n_neighbors=1, algorithm='brute')
# train the classifier 
classifier.fit(df)
# run the classifier on all the data 
distance, index = classifier.kneighbors(edf)
#check the best match
distance.min()

0.0

# Brute force seems to be the best algorithm 

I'm going to line up the results and see what happens

In [ ]:
# merge distance and index into one array, then a DF
dist_df = pd.DataFrame(np.concatenate((distance, index), axis=1), columns=['distance', 'USGS_index'])
dist_df = dist_df.set_index(pd.Index(np.arange(1,20001)))

In [ ]:
# merge the predictions back with the quake data
df2 = edf.reset_index().merge(dist_df.reset_index(), left_on=0, right_on='index')

In [ ]:
#seperate everything with 0 distance, theoretically they are the same quake
zero_dist = df2[df2['distance'] == 0]
zero_dist.shape, low_dist.shape

In [ ]:
# rename columns for ease of reading
zero_dist = zero_dist.rename(columns={0: 'E_id', 1:'E_time', 2:'E_lat', 3:'E_lon', 4:'E_mag', 'index':'id(also)', 'distance':'distance_from_USGS', 'USGS_index':'USGS_id'})
low_dist = low_dist.rename(columns={0: 'E_id', 1:'E_time', 2:'E_lat', 3:'E_lon', 4:'E_mag', 'index':'id(also)', 'distance':'distance_from_USGS', 'USGS_index':'USGS_id'})
df2 = df2.rename(columns={0: 'E_id', 1:'E_time', 2:'E_lat', 3:'E_lon', 4:'E_mag', 'index':'id(also)', 'distance':'distance_from_USGS', 'USGS_index':'USGS_id'})

In [ ]:
#checking to see if any match up, it doesn't look like they do 
merged = zero_dist.merge(df.reset_index(), left_on='USGS_id', right_on=0, how='left')
merged['time_matches'] = merged['E_mag'] == merged[4]
merged[merged['time_matches']]

Thats weird. I would have really expected that to work. I'm going to see what I can do with the kneighbors algorithm to get better results

In [ ]:
#I have no idea how this works 
classifier.radius_neighbors(edf)

## Manual matching 
Well, this seems rediculous but I'm not having a lot of luck with matching up quakes as is, I'm going to try writin an algorithm that takes a quake from EMSC, then gets all quakes within 10 seconds in either direction and then matches up the magnitudes and locations. 

In [13]:
for i, quake in enumerate(EMSC_QUAKES[:1000]):
    replies = query_all(f'SELECT * FROM USGS WHERE Time BETWEEN {quake[1]-10000} AND {quake[1]+10000}')
    if len(replies) > 0:
        for reply in replies:
            if np.absolute(reply[5]-quake[4]) < .5:
                print('quake', quake)
                print('reply', reply)
                print('-------------')
            

quake (60, 1586213070700, 47.53, 9.27, 0.7)
reply (22502, '16km ESE of Anza, CA', 1586213068900, 33.5075, -116.5156667, 0.68, False)
-------------
quake (78, 1586196678500, 38.98, 27.89, 2.2)
reply (22402, '8km SSE of Pahala, Hawaii', 1586196670910, 19.1359997, -155.4389954, 2.0, False)
-------------
quake (145, 1586172120400, 44.41, -115.17, 2.5)
reply (22220, '8km ENE of Pahala, Hawaii', 1586172113950, 19.2238331, -155.4018402, 2.29, False)
-------------
quake (168, 1586163540000, -5.79, 102.53, 3.0)
reply (22149, '3km SE of Guanica, Puerto Rico', 1586163548110, 17.9506, -66.8801, 2.92, False)
-------------
quake (283, 1586129985800, 17.95, -66.87, 2.9)
reply (21848, '6km NE of Magna, Utah', 1586129993040, 40.751667, -112.0468369, 2.65, False)
-------------
quake (317, 1586120498700, 35.87, -117.71, 2.1)
reply (21672, '11km NE of Coso Junction, CA', 1586120504470, 36.109333, -117.8496704, 1.76, False)
-------------
quake (317, 1586120498700, 35.87, -117.71, 2.1)
reply (21673, '12km N

TypeError: object of type 'NoneType' has no len()

OK so that was a little slow. I think I'm going to try to use the dataframe that I already have and replicate the search using pandas

In [8]:
for i, quake in enumerate(EMSC_QUAKES[:1]):
    replies = df[(df[1] > quake[1]-10000) & df[1] < quake[1]+10000]
    if len(replies) > 0:
        for reply in replies.values:
            print(reply)
            if np.absolute(reply[3]-quake[4]) < .3:
                print('quake', quake)
                print('reply', reply)
                print('-------------')

[ 1.58372159e+12  5.63985000e+01 -1.57258000e+02  2.60000000e+00]
quake (1, 1586237049900, 37.8, 16.91, 2.9)
reply [ 1.58372159e+12  5.63985000e+01 -1.57258000e+02  2.60000000e+00]
-------------
[ 1.58372117e+12  6.22701000e+01 -1.52285100e+02  2.30000000e+00]
[ 1.58372013e+12  3.57166667e+01 -1.17489333e+02  1.25000000e+00]
[ 1.58372008e+12  1.79238000e+01 -6.67561000e+01  2.10000000e+00]
[ 1.58371973e+12  5.97152000e+01 -1.52588900e+02  1.00000000e+00]
[ 1.58371954e+12  6.24082000e+01 -1.51423300e+02  1.70000000e+00]
[ 1.58371951e+12  3.56268333e+01 -1.17464667e+02  2.26000000e+00]
[ 1.58371904e+12  3.35150000e+01 -1.16456667e+02  6.20000000e-01]
[ 1.58371862e+12  3.33606667e+01 -1.16308333e+02  8.90000000e-01]
[ 1.5837184e+12  6.2200400e+01 -1.5118130e+02  2.6000000e+00]
quake (1, 1586237049900, 37.8, 16.91, 2.9)
reply [ 1.5837184e+12  6.2200400e+01 -1.5118130e+02  2.6000000e+00]
-------------
[ 1.58371738e+12  3.87801666e+01 -1.22721832e+02  7.10000000e-01]
[ 1.58371722e+12  3.8764

KeyboardInterrupt: 

In [22]:
len(df)

23456

In [20]:
EMSC_QUAKES[0]

(1, 1586237049900, 37.8, 16.91, 2.9)